In [6]:
docs = ['AI ', 'Travel']
query = 'AI big data'

In [9]:
from gensim.summarization.bm25 import BM25
corpus = ["The little fox ran home",
          "dogs are the best ",
          "Yet another doc ",
          "I see a little fox with another small fox",
          "last doc without animals"]

def simple_tok(sent:str):
    return sent.split()

tok_corpus = [simple_tok(s) for s in corpus]
bm25 = BM25(tok_corpus)
query = "a little fox".split()
scores = bm25.get_scores(query)

best_docs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:3]
for i, b in enumerate(best_docs):
    print(f"rank {i+1}: {corpus[b]}")

rank 1: I see a little fox with another small fox
rank 2: The little fox ran home
rank 3: dogs are the best 


In [10]:
scores

[0.6729444732424259, 0.0, 0.0, 1.437564024113848, 0.0]

In [31]:
from gensim.summarization.bm25 import BM25

class FastBM25:
    def __init__(self, corpus=list, batch_size=32):
        
        # word segmentation
        self.corpus_ws = self.word_segmentation(corpus=corpus, batch_size=batch_size)
        
        # lauch bm25 moidel
        self.bm25 = BM25(self.corpus_ws)


    def word_segmentation(self, corpus=list ,batch_size=int):
        # init parameter and container
        corpus_ws = []
        batch_num = int(len(corpus) / batch_size) + 1
        # main
        for i in range(batch_num):
            batch_corpus = corpus[i*batch_size : (i+1)*batch_size]
            batch_corpus_ws = self.word_segmentation_func(batch_corpus=batch_corpus)
            corpus_ws += batch_corpus_ws
        return corpus_ws
    
    def word_segmentation_func(self, batch_corpus=list):
        batch_corpus_ws = batch_corpus
        batch_corpus_ws = [sent.split() for sent in batch_corpus]
        return batch_corpus_ws
    
    
    def main(self, query=str):
        query_ws = self.word_segmentation_func(batch_corpus=[query])[0]
        scores = self.bm25.get_scores(query_ws)
        corpsu_with_score = [[sent_ws, scores[i]] for i, sent_ws in enumerate(self.corpus_ws)]
        corpsu_with_score = sorted(corpsu_with_score, reverse=True, key=lambda x:x[1])
        return corpsu_with_score
        

In [32]:
corpus = ["The little fox ran home",
          "dogs are the best ",
          "Yet another doc ",
          "I see a little fox with another small fox",
          "last doc without animals"]

fast_bm25_model = FastBM25(corpus=corpus, batch_size=3)
fast_bm25_model.main(query="a little fox")

[[['I', 'see', 'a', 'little', 'fox', 'with', 'another', 'small', 'fox'],
  1.437564024113848],
 [['The', 'little', 'fox', 'ran', 'home'], 0.6729444732424259],
 [['dogs', 'are', 'the', 'best'], 0.0],
 [['Yet', 'another', 'doc'], 0.0],
 [['last', 'doc', 'without', 'animals'], 0.0]]